In [94]:
import clickhouse_connect
import pandas as pd
import numpy as np
from IPython.core.completerlib import import_re

# if __name__ == '__main__':
#     client = clickhouse_connect.get_client(
#         host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
#         user='default',
#         password='3phq~SvlfTr39',
#         secure=True
#     )
#     print("Result:", client.query("SELECT 1").result_set[0][0])

In [44]:
import os

def get_csv_files_info(folder_path):
    """
    Получает названия и полные пути всех CSV-файлов в указанной папке.

    Args:
        folder_path (str): Путь к папке.

    Returns:
        list: Список словарей, где каждый словарь содержит информацию о CSV-файле:
              {'name': 'имя_файла.csv', 'path': '/полный/путь/к/файлу/имя_файла.csv'}
              Возвращает пустой список, если папка не существует или не содержит CSV-файлов.
    """

    csv_files = []

    # Проверяем, существует ли папка
    if not os.path.exists(folder_path):
        print(f"Ошибка: Папка '{folder_path}' не существует.")
        return csv_files  # Возвращаем пустой список

    # Перебираем все файлы и папки в указанной директории
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)

        # Проверяем, является ли элемент файлом и имеет ли расширение .csv
        if os.path.isfile(item_path) and item.lower().endswith(".csv"):
            csv_files.append({'name': item, 'path': item_path})

    return csv_files

# Пример использования:
if __name__ == '__main__':
    folder_path = r"C:\Users\Misha\PycharmProjects\click_power\test\final_tabels"  # Замените на свой путь

    csv_files_info = get_csv_files_info(folder_path)

    if csv_files_info:
        print("CSV-файлы в папке:")
        for file_info in csv_files_info:
            print(f"  Имя: {file_info['name']}")
            print(f"  Путь: {file_info['path']}")
    else:
        print("В папке нет CSV-файлов, или папка не существует.")

CSV-файлы в папке:
  Имя: заказы.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\заказы.csv
  Имя: каналы.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\каналы.csv
  Имя: клиенты.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\клиенты.csv
  Имя: менеджеры.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\менеджеры.csv
  Имя: промокоды.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\промокоды.csv
  Имя: склады.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\склады.csv
  Имя: статусы_заказов.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\статусы_заказов.csv
  Имя: типы_доставки.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\типы_доставки.csv
  Имя: типы_оплаты.csv
  Путь: C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\типы_оплаты.csv
  Имя: товары.csv
  Путь: C:\Users\Misha\Py

In [45]:
client = clickhouse_connect.get_client(
        host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
        user='default',
        password='3phq~SvlfTr39',
        secure=True
    )

In [123]:
def is_date_like(value):
    """Проверяет, похожа ли строка на дату (формат YYYY-MM-DD)."""
    try:
        # Попытка преобразовать строку в дату
        pd.to_datetime(value, format='%Y-%m-%d', errors='raise')
        return True
    except: #(ValueError, TypeError)
        return False

def create_database(client, database_name):
    """Создает базу данных в ClickHouse, если она еще не существует."""
    try:
        client.command(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        print(f"База данных '{database_name}' успешно создана (если ее не было).")
    except Exception as e:
        print(f"Ошибка при создании базы данных '{database_name}': {e}")

def get_clickhouse_type(pandas_dtype):
    """Преобразует тип данных Pandas в соответствующий тип данных ClickHouse."""
    if pandas_dtype == 'int64':
        return 'Int64'  # Или Int32, UInt32, в зависимости от диапазона значений
    elif pandas_dtype == 'float64':
        return 'Float64'
    elif pandas_dtype == 'bool':
        return 'Bool'
    elif pandas_dtype == 'datetime64[ns]':
        return 'DateTime'
    else:
        return 'String'  # По умолчанию используем String


def get_column_types_for_clickhouse(filepath, encoding='utf-16'):
    """Определяет типы столбцов, проверяя первую строку на формат даты."""
    try:
        df = pd.read_csv(filepath, encoding=encoding)
         # Явное преобразование всех object столбцов в string
        for col in df.select_dtypes(include=['object']).columns:
            # Замена NaN и None на пустые строки
            df[col] = df[col].fillna('').astype(str)

        # Автоматическое определение столбцов с датами на основе первой строки
        for column in df.columns:
            if is_date_like(str(df[column][0])): # Проверяем первую строку
                try:
                    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d', errors='raise')
                # except (ValueError, TypeError) as e:
                #     print(f"Не удалось преобразовать столбец '{column}' в дату: {e}")
                except:
                    pass

        column_types = {}
        for column in df.columns:
            pandas_dtype = str(df[column].dtype)
            clickhouse_type = get_clickhouse_type(pandas_dtype)
            column_types[column] = clickhouse_type
        return column_types

    except Exception as e:
        print(f"Ошибка при определении типов данных: {e}")
        return None

def create_clickhouse_table(client, database, table_name, table_schema):
    """
    Создает таблицу в ClickHouse, если она не существует.

    Args:
        client: Объект клиента ClickHouse.
        database: Имя базы данных.
        table_name: Имя таблицы.
        table_schema: Строка, содержащая SQL-запрос для создания таблицы.
    """

    try:
        client.command(f"USE {database}")

        # Удаляем таблицу, если она существует
        client.command(f"DROP TABLE IF EXISTS {database}.{table_name}")
        print(f"Удалена таблица '{table_name}' в базе данных '{database}', если она существовала.")

        # Проверяем, существует ли таблица
        result = client.query(f"EXISTS TABLE {database}.{table_name}")
        # Проверяем, существует ли таблица
        table_exists = result.result[0][0] if hasattr(result, 'result') and result.result else False

        if not table_exists:
            client.command(table_schema)
            print(f"Таблица '{table_name}' успешно создана в базе данных '{database}'.")
        else:
            print(f"Таблица '{table_name}' уже существует в базе данных '{database}'.")

    except Exception as e:
        print(f"Ошибка при создании таблицы '{table_name}': {e}")

def insert_data_from_csv(client, database, table_name, csv_filepath, encoding='utf-16'):
    """Вставляет данные из CSV-файла в таблицу ClickHouse."""
    try:
        df = pd.read_csv(csv_filepath, encoding=encoding) #или cp1251, если utf-16 не подходит

        # Заменяем все NaN значения пустой строкой и меняем тип object на string
        for col in df.columns:
            if df[col].dtype == 'object':  # Проверяем, является ли столбец строковым
                df[col] = df[col].fillna('').astype(str)  # Заменяем NaN на пустые строки
            elif df[col].dtype == 'float64':  # Проверяем, является ли столбец float
                 df[col] = df[col].replace([np.nan, np.inf, -np.inf], 0)  #Заменяем float на 0

        # Заменяем пустые строки на "unknown"
        df = df.replace(r'^\s*$', 'пусто', regex=True) #Заменяем пустые строки

        # Автоматическое определение столбцов с датами на основе первой строки
        for column in df.columns:
            if is_date_like(str(df[column][0])): # Проверяем первую строку
                try:
                    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d', errors='raise')
                except:
                    pass

        print(df.info())
        print(df.head(2))
        # Убедимся, что имена столбцов в DataFrame соответствуют именам столбцов в таблице ClickHouse.
        #При необходимости переименуйте столбцы DataFrame:
        #df.rename(columns={'старое_имя': 'новое_имя', ...}, inplace=True)

        client.insert_df(f'{database}.{table_name}', df)
        print(f"Данные из CSV-файла '{csv_filepath}' успешно вставлены в таблицу '{database}.{table_name}'.")

    except Exception as e:
        print(df.info())
        print(df.head(2))
        print(f"Ошибка при вставке данных из CSV-файла: {e}")

In [124]:
encoding = 'utf-16'

filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\заказы.csv'
table_schema = 'dashminox'
table_name = 'orders'
key_column = 'row_number_key'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\каналы.csv'
# table_schema = 'dashminox'
# table_name = 'channels'
# key_column = 'КаналId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\клиенты.csv'
# table_schema = 'dashminox'
# table_name = 'clients'
# key_column = 'ClientId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\менеджеры.csv'
# table_schema = 'dashminox'
# table_name = 'managers'
# key_column = 'МенеджерId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\промокоды.csv'
# table_schema = 'dashminox'
# table_name = 'promocodes'
# key_column = 'ПромокодId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\склады.csv'
# table_schema = 'dashminox'
# table_name = 'warehouses'
# key_column = 'СкладId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\статусы_заказов.csv'
# table_schema = 'dashminox'
# table_name = 'order_status'
# key_column = 'СтатусЗаказаId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\типы_доставки.csv'
# table_schema = 'dashminox'
# table_name = 'delivery_type'
# key_column = 'ТипДоставкиId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\типы_оплаты.csv'
# table_schema = 'dashminox'
# table_name = 'payment_type'
# key_column = 'ТипОплатыId'

# filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\товары.csv'
# table_schema = 'dashminox'
# table_name = 'products'
# key_column = 'sku_clear'



column_types = get_column_types_for_clickhouse(filepath, encoding)

if column_types:
    print("Типы столбцов для ClickHouse:")
    for column, data_type in column_types.items():
        print(f"{column}: {data_type}")

    # Генерация SQL-запроса для создания таблицы (пример)
    create_table_statement = f"CREATE TABLE IF NOT EXISTS {table_schema}.{table_name} ("
    for column, data_type in column_types.items():
       #Обрамляем названия столбцов, содержащие пробелы или русские буквы
        escaped_column = f"`{column}`" if ' ' in column or any(ord(c) > 127 for c in column) else column
        create_table_statement += f"\n    {escaped_column} {data_type},"
    create_table_statement = create_table_statement.rstrip(",")  # Удаляем последнюю запятую
    if key_column:
         create_table_statement += f"\n) ENGINE = MergeTree()\nORDER BY `{key_column}`"
    else:
        create_table_statement += f"\n) ENGINE = MergeTree()\nORDER BY 'UID'"
    print("\nSQL-запрос для создания таблицы:\n", create_table_statement)

Типы столбцов для ClickHouse:
Номер: String
sku_clear: String
date: DateTime
hour: Float64
day: Float64
day_of_week: String
Количество_полное: Float64
Выручка: Float64
Себестоимость: Float64
Маржа: Float64
Оплачено: Float64
Итоговая скидка: Float64
Себестоимость доставки: Float64
Стоимость доставки: Float64
Наложенный платёж: String
Тип оплаты: String
Промокод: String
Тип доставки: String
Внутренний ID: Float64
Передача данных в службу доставки: String
Статус заказа: String
Склад: String
Первый заказ: String
Последний заказ: String
Порядковый номер заказа у клиента: Float64
Способ оформления: String
Контактный телефон: String
Менеджер: String
Причина отмены заказа: String
Канал: String
ClientId: Int64
МенеджерId: Int64
КаналId: Int64
ТипОплатыId: Int64
ПромокодId: Int64
ТипДоставкиId: Int64
СтатусЗаказаId: Int64
СкладId: Int64
row_number_key: Int64

SQL-запрос для создания таблицы:
 CREATE TABLE IF NOT EXISTS dashminox.orders (
    `Номер` String,
    sku_clear String,
    date DateTim

In [125]:
# Создаем базу данных, если ее нет
create_database(client, database_name)
# создаем таблицу
create_clickhouse_table(client, database_name, table_name, create_table_statement)
# Вставляем данные из CSV-файла
insert_data_from_csv(client, database_name, table_name, filepath, encoding)

База данных 'dashminox' успешно создана (если ее не было).
Удалена таблица 'orders' в базе данных 'dashminox', если она существовала.
Таблица 'orders' успешно создана в базе данных 'dashminox'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39156 entries, 0 to 39155
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Номер                              39156 non-null  object        
 1   sku_clear                          39156 non-null  object        
 2   date                               39156 non-null  datetime64[ns]
 3   hour                               39156 non-null  float64       
 4   day                                39156 non-null  float64       
 5   day_of_week                        39156 non-null  object        
 6   Количество_полное                  39156 non-null  float64       
 7   Выручка                            39156 non-null  fl

In [38]:
import clickhouse_connect

def create_database(client, database_name):
    """Создает базу данных в ClickHouse, если она еще не существует."""
    try:
        client.command(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        print(f"База данных '{database_name}' успешно создана (если ее не было).")
    except Exception as e:
        print(f"Ошибка при создании базы данных '{database_name}': {e}")


def create_clickhouse_table(client, database, table_name, table_schema):
    """
    Создает таблицу в ClickHouse, если она не существует.

    Args:
        client: Объект клиента ClickHouse.
        database: Имя базы данных.
        table_name: Имя таблицы.
        table_schema: Строка, содержащая SQL-запрос для создания таблицы.
    """

    try:
        client.command(f"USE {database}")

        # Проверяем, существует ли таблица
        result = client.query(f"EXISTS TABLE {database}.{table_name}")
        table_exists = result.result[0][0] if hasattr(result, 'result') and result.result else False

        if not table_exists:
            client.command(table_schema)
            print(f"Таблица '{table_name}' успешно создана в базе данных '{database}'.")
        else:
            print(f"Таблица '{table_name}' уже существует в базе данных '{database}'.")

    except Exception as e:
        print(f"Ошибка при создании таблицы '{table_name}': {e}")

# Пример использования:
if __name__ == '__main__':
    client = clickhouse_connect.get_client(
        host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
        user='default',
        password='3phq~SvlfTr39',
        secure=True
    )

    database_name = 'dashminox'
    table_name = 'payment_type' # Укажите имя таблицы

    # Создаем базу данных, если ее нет
    create_database(client, database_name)

    # Определяем схему таблицы
    table_schema = f"""
    CREATE TABLE IF NOT EXISTS {database_name}.{table_name} (
        `Тип оплаты` Nullable(String),
        `ТипОплатыId` UInt8
    ) ENGINE = MergeTree()
    ORDER BY `ТипОплатыId`
    """
    # создаем таблицу
    create_clickhouse_table(client, database_name, table_name, table_schema)

База данных 'dashminox' успешно создана (если ее не было).
Таблица 'payment_type' успешно создана в базе данных 'dashminox'.


In [100]:
# Удаление таблицы

table_name = 'orders'
database_name = 'dashminox'

try:
    client.command(f"DROP TABLE IF EXISTS {database_name}.{table_name}")
    print(f"Таблица '{table_name}' успешно удалена из базы данных '{database_name}'.")
except Exception as e:
    print(f"Ошибка при удалении таблицы '{table_name}': {e}")

Таблица 'orders' успешно удалена из базы данных 'dashminox'.
